In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
input_shape = (28,28,1)

x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test=x_test/255.0

In [ ]:
# y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
# y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [ ]:
# latent_dim = 64 

# class Autoencoder(Model):
#   def __init__(self, latent_dim):
#     super(Autoencoder, self).__init__()
#     self.latent_dim = latent_dim   
#     self.encoder = tf.keras.Sequential([
#       layers.Flatten(),
#       layers.Dense(128, activation='relu'),
#       layers.Dense(latent_dim, activation='relu'),
#     ])
#     self.decoder = tf.keras.Sequential([
#       layers.Dense(128, activation='relu'),
#       layers.Dense(784, activation='sigmoid'),
#       layers.Reshape((28,28,1))
#     ])

#   def call(self, x):
#     encoded = self.encoder(x)
#     decoded = self.decoder(encoded)
#     return decoded
  
# autoencoder = Autoencoder(latent_dim) 

In [ ]:
latent_dim = 64
class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(28,28,1)),
      layers.MaxPooling2D((2, 2), padding='same'),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same'),   
      layers.MaxPooling2D((2, 2), padding='same'),
      layers.Conv2D(4, (3, 3), activation='relu', padding='same'), 
      layers.MaxPooling2D((2, 2), padding='same'),
       
      # layers.BatchNormalization(),    
      # layers.UpSampling2D(),
      # layers.Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu'),   
      # layers.BatchNormalization(),
      # layers.Conv2D(1,  kernel_size=32,padding='same', activation='sigmoid'),
      
    ])
    self.decoder = tf.keras.Sequential([
      # layers.Conv2D(1, kernel_size=32, activation='relu', padding='same'),
      layers.Conv2D(4, (3, 3), activation='relu', padding='same'),
      layers.UpSampling2D((2, 2)),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
      layers.UpSampling2D((2, 2)),
      layers.Conv2D(16, (3, 3), activation='relu'),
      layers.UpSampling2D((2, 2)), 
      layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same'),
      layers.Reshape((28,28,1))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim) 

In [ ]:
batch_size = 32
num_classes = 10
epochs = 20

In [ ]:
for m in range(20,21,2):
  num_nodes = m
  each_dataset = int(x_train.shape[0]/num_nodes)
  print(each_dataset)
  x_train_list = []
  history_list = []
  autoencoder_list = []
  encoded_images_train = []
  decoded_images_train = []
  encoded_images_test = []
  decoded_images_test = []

  for i in range(num_nodes):
    x_train_list.append(x_train[int(i*each_dataset):int((i+1)*each_dataset)])

    autoencoder_list.append(Autoencoder(latent_dim)) 
    autoencoder_list[i].compile(optimizer='adam', loss=losses.MeanSquaredError())
    print('Node Number',i)
    autoencoder_list[i].fit(x_train_list[i], x_train_list[i],
                  epochs=10,batch_size = 32,shuffle=True)
    
    encoded_images_train.append(autoencoder_list[i].encoder(x_train_list[i]).numpy())
    # decoded_images_train.append(autoencoder_list[i]r.decoder(encoded_images_train[i]).numpy())
    # encoded_images_test.append(autoencoder_list[i].encoder(x_test).numpy())
    # decoded_images_test.append(autoencoder_list[i].decoder(encoded_images_test[i]).numpy())

    n = 10
    plt.figure(figsize=(20, 4))
    for j in range(n):
      # display original
      ax = plt.subplot(2, n, j + 1)
      plt.imshow(x_test[j,:,:,0])
      plt.title("original")
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)

      # display reconstruction
      ax = plt.subplot(2, n, j + 1 + n)
      plt.imshow(encoded_images_test[i].reshape(encoded_images_test[i].shape[0],8,8,1)[j,:,:,0])
      plt.title("reconstructed")
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
    plt.show()
  overall_encoded_train = np.zeros((x_train.shape[0],8,8,1))
  # overall_encoded_test = np.zeros((x_test.shape[0],8,8,1))

  for k in range(num_nodes):
    overall_encoded_train[int(k*each_dataset):int((k+1)*each_dataset)] = encoded_images_train[k].reshape(encoded_images_train[k].shape[0],8,8,1)

  overall_encoded_train = overall_encoded_train.reshape((overall_encoded_train.shape[0], 8,8, 1))

  # model2 = tf.keras.models.Sequential([
  #     tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu', input_shape=(8,8,1)),
  #     tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu'),
  #     tf.keras.layers.MaxPool2D(),
  #     tf.keras.layers.Dropout(0.25),
  #     tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
  #     tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
  #     tf.keras.layers.MaxPool2D(strides=(2,2)),
  #     tf.keras.layers.Dropout(0.25),
  #     tf.keras.layers.Flatten(),
  #     tf.keras.layers.Dense(128, activation='relu'),
  #     tf.keras.layers.Dropout(0.5),
  #     tf.keras.layers.Dense(num_classes, activation='softmax')
  # ])

  # model2.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

  # history = model2.fit(overall_encoded_train, y_train,
  #                     batch_size=batch_size,
  #                     epochs=20,
  #                     validation_split=0.1)

  # for l in range(num_nodes):
  #   test_loss, test_acc = model2.evaluate(encoded_images_test[l].reshape(10000,8,8,1), y_test)
  # saving_dict[m] = history.history

In [ ]:
import pickle
with open('overall_encoded_train_20.txt','wb') as fp:
  pickle.dump(overall_encoded_train,fp)

In [ ]:
!git clone https://github.com/stijani/tutorial

In [ ]:
cd tutorial

In [ ]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

from fl_mnist_implementation_tutorial_utils import *

In [ ]:
# mnist = tf.keras.datasets.mnist
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# # idx = np.argsort(y_train)
# # x_train = x_train[idx]
# # y_train = y_train[idx]
# x = np.concatenate((x_train, x_test))
# y = np.concatenate((y_train, y_test))

# train_size = 0.7
X_train, X_test, y_train, y_test = train_test_split(overall_encoded_train, y_train, test_size=0.15, random_state=42)
# X_train = overall_encoded_train
# X_test = overall_encoded_test
# idx = np.argsort(y_train)
# X_train = X_train[idx]
# y_train = y_train[idx]

input_shape = (8,8, 1)

X_train=X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
# X_train=X_train / 255.0
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
# X_test=X_test/255.0
# y_test = np.concatenate((y_test, np.tile(y_test,19)))
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [ ]:
clients = create_clients(X_train, y_train, num_clients=2, initial='client')

In [ ]:
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [ ]:
num_classes = 10
input_shape = (8,8,1)

In [ ]:
class SimpleMLP:
    @staticmethod
    def build(classes):
        model = Sequential()
        # model.add(Conv2D(32, (5,5), padding='same', activation='relu', input_shape=input_shape))
        # model.add(MaxPool2D())
        # model.add(Dropout(0.25))
        # model.add(Flatten())
        # model.add(Dense(128, activation='relu'))
        # model.add(Dropout(0.5))
        # model.add(Dense(classes, activation='softmax'))

        model.add(Conv2D(32, (5,5), padding='same', activation='relu', input_shape=input_shape))
        model.add(Conv2D(32, (5,5), padding='same', activation='relu'))
        model.add(MaxPool2D())
        model.add(Dropout(0.25))
        model.add(Conv2D(64, (5,5), padding='same', activation='relu'))
        model.add(Conv2D(64, (5,5), padding='same', activation='relu'))
        model.add(MaxPool2D(strides=(2,2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(classes, activation='softmax'))
        return model

In [ ]:
lr = 0.01 
comms_round = 50
loss='categorical_crossentropy'
metrics = ['accuracy']
# optimizer=tf.keras.optimizers.Adam()
optimizer = SGD(learning_rate=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )            

In [ ]:
list3 = []
list4 = []

In [ ]:
smlp_global = SimpleMLP()
global_model = smlp_global.build(10)
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(10)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        history = local_model.fit(clients_batched[client], epochs=1)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, y_test, global_model, comm_round)

        # print(Y_test)
        list3.append(global_acc)
        list4.append(global_loss)